### WikiEngine

In [5]:
%run wikidump.py

path_base = '/Users/harangju/Developer/data/wiki/'
name_xml = 'enwiki-20190801-pages-articles-multistream.xml.bz2'
name_index = 'enwiki-20190801-pages-articles-multistream-index.txt.bz2'
path_xml = path_base + name_xml
path_index = path_base + name_index
dump = WikiDump(path_xml, path_index)

### Exploring the wiki dump

In [6]:
dump.load_page('Portal:Physics/Topics')
dump.links[:5]

WikiDump: Loading index...
WikiDump: Loaded.


['Classical physics', 'mechanics', 'optics', 'electricity', 'magnetism']

In [7]:
dump.load_page('Danielle Bassett')
dump.links[:5]

['University of Pennsylvania',
 'Pennsylvania State University',
 'University of Cambridge',
 'Sloan Research Fellowship',
 'MacArthur fellowship']

In [8]:
dump.load_page('Matter', filter_top=True)
dump.links[5:10]

['plasma (physics)', 'quartz', 'solid', 'water', 'liquid']

In [9]:
dump.load_page('Index of physics articles (P)')
dump.links[50:54]

['Pandemonium effect', 'Panemone windmill', 'Panofsky Prize', 'Pantur Silaban']

### Wikipedia hypernet traversal

In [10]:
import networkx as nx

class WikiCrawler():
    @staticmethod
    def bfs(graph, dump, queue, depth_goal=1):
        depth = 0
        depth_num_items = len(queue)
        depth_inc_pending = False
        while queue:
            page = queue.pop(0)
            depth_num_items -= 1
            if depth_num_items == 0:
                depth += 1
                print('Depth: ' + str(depth))
                depth_inc_pending = True
            if dump.load_page(page, filter_top=True):
                for link in dump.links:
                    link = str(link).split('#')[0].capitalize()
                    if (page, link) not in graph.edges:
                        graph.add_edge(link, page)
                        queue.append(link)
            if depth_inc_pending:
                depth_num_items = len(queue)
                depth_inc_pending = False
            if depth == depth_goal:
                break

In [11]:
graph = nx.DiGraph()
queue = ['Matter']
WikiCrawler.bfs(graph, dump, queue, 3)

Depth: 1
Depth: 2
Depth: 3


In [12]:
len(graph.nodes)

9492

### Port to gephi

In [13]:
nx.write_gexf(graph, path_base + 'graphs/matter_d3.gexf')

### Edge weights